In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
from toolbox import sphereical_transport
#inputs
n = 4
cells = 200
r = 1.0
scat_order = 0
sig_t = 1.0
sig_a = 1.0
q_0 = np.zeros(cells) #p/cm^3-s
BC = 'vac'
DSA = False
legendre_xs_mom = np.array([0])
#generate angular grid center points (roots of legendre polynomial of degree n) and the
#corresponding gauss quadrature weights
ang_cell_centers, w = np.polynomial.legendre.leggauss(n)
#generate angular cell edge values 
ang_cell_edges = np.zeros(n+1)
ang_cell_edges[0] = -1
for i in range(1,n+1):
    ang_cell_edges[i] = ang_cell_edges[i-1] + w[i-1]
#generate alpha coefficients
alpha = np.zeros(n+1)
for i in range(1,n+1):
    alpha[i] = alpha[i-1] - (2*w[i-1]*ang_cell_centers[i-1])
alpha[n] = 0
#generate beta coefficients
beta = np.zeros(n)
for i in range(n):
    beta[i] = (ang_cell_centers[i] - ang_cell_edges[i])/(ang_cell_edges[i+1] - ang_cell_edges[i])
#initialize sphereical transport class
sph_trans = sphereical_transport(q_0,legendre_xs_mom,cells,n,r,sig_t,sig_a,BC,scat_order,ang_cell_centers,w,alpha,beta)
#initial guess for source
ang_flux_0 = np.zeros(n)
for i in range(cells):
    sph_trans.update_scat_source_mom(i,ang_flux_0)
#generate initial phi using initial guess for psi
phi_0 = np.zeros(cells)
for i in range(cells):
    for n in range(np.size(ang_flux_0)):
        phi_0[i] += ang_flux_0[n]*w[n]
epsilon = 1
#initialize spatial inflow vector
spatial_inflow = np.zeros(n)
#begin source iteration
while epsilon > 1e-6:
    #compute psi at mu = -1 which will be the first angular inflow vector
    ang_inflow, origin_flux = sph_trans.starting_direction()
    #do the full domain sweep
    phi,outflow = sph_trans.sweep(ang_inflow,origin_flux)
    if DSA:
        DSA_acc(phi,phi_0)
    epsilon = np.linalg.norm(phi_0 - phi)
    phi_0 = phi
print(phi)

[0.73616432 0.73646495 0.73683361 0.73723821 0.73765836 0.73809117
 0.73853494 0.73898902 0.73945309 0.73992703 0.7404108  0.7409044
 0.74140787 0.74192128 0.7424447  0.74297822 0.74352192 0.74407591
 0.74464029 0.74521516 0.74580063 0.74639681 0.74700382 0.74762178
 0.7482508  0.74889101 0.74954252 0.75020547 0.75087998 0.75156617
 0.75226419 0.75297416 0.75369621 0.75443048 0.75517712 0.75593625
 0.75670803 0.75749259 0.75829007 0.75910064 0.75992443 0.76076159
 0.76161228 0.76247666 0.76335487 0.76424708 0.76515345 0.76607414
 0.76700931 0.76795914 0.76892378 0.76990341 0.77089821 0.77190834
 0.77293399 0.77397533 0.77503255 0.77610583 0.77719536 0.77830132
 0.77942392 0.78056333 0.78171977 0.78289342 0.7840845  0.7852932
 0.78651974 0.78776432 0.78902717 0.79030849 0.7916085  0.79292744
 0.79426553 0.795623   0.79700008 0.79839702 0.79981405 0.80125142
 0.80270939 0.8041882  0.80568812 0.8072094  0.80875233 0.81031716
 0.81190419 0.8135137  0.81514597 0.8168013  0.81848    0.820182

In [5]:
outflows = [0.16595038478084856,0.16595141023323273,0.1659516649517479]
order = np.abs((outflows[1]-outflows[0])/(outflows[1]-outflows[2]))
print(order)

4.025825855229543
